In [1]:
! pip install xgboost

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import xgboost as xgb

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
dataset=pd.read_csv("insurance_pre.csv")

In [5]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [6]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [7]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [8]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [9]:
independent=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]

In [10]:
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [11]:
dependent=dataset[['charges']]

In [12]:
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [23]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': [500], 'max_depth' : [4], 'min_samples_split' : [5], 'learning_rate' : [0.01], 'loss' : ['squared_error']}    

In [25]:
grid = GridSearchCV(XGBRegressor(), param_grid, refit=True, verbose=3, n_jobs=-1)
grid.fit(independent,dependent)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


C:\Users\sathe\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [12:03:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "loss", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bi...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01], 'loss': ['squared_error'],
                         'max_depth': [4], 'min_samples_split': [5],
                         'n_estimators': [500]},
             verbose=3)

In [27]:
re=grid.cv_results_
print("The R_score value for the best parameter{}:".format(grid.best_params_))

The R_score value for the best parameter{'learning_rate': 0.01, 'loss': 'squared_error', 'max_depth': 4, 'min_samples_split': 5, 'n_estimators': 500}:


In [28]:
table=pd.DataFrame.from_dict(re)

In [29]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_loss,param_max_depth,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,3.552487,1.401695,0.034375,0.02296,0.01,squared_error,4,5,500,"{'learning_rate': 0.01, 'loss': 'squared_error...",0.878869,0.809542,0.895725,0.850679,0.866647,0.860292,0.029356,1


In [30]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age: 0
BMI: 0
Children: 0
Sex Male 0 or 1: 0
Smoker Yes 0 or 1: 0


In [33]:
Future_Prediction=grid.predict([[age_input, bmi_input, children_input, sex_male_input, smoker_yes_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[2445.3354]


In [36]:
import pickle
filename="finalized_model_XgboostRegressor_GridSearchCV.sav"
pickle.dump(grid,open(filename, 'wb'))

In [38]:
loaded_model=pickle.load(open("finalized_model_XgboostRegressor_GridSearchCV.sav", 'rb'))
result=loaded_model.predict([[0,0,0,0,0]])

In [39]:
result

array([2445.3354], dtype=float32)